# 👂 Active learning for text classification with small-text

In this tutorial, you will learn how to set up a complete active learning loop with a [Hugging Face transformer](https://huggingface.co/docs/transformers/index):

 - Use the excellent [small-text](https://github.com/webis-de/small-text) library to set up your active learner;
 - Use a [Argilla listener](../reference/python/python_listeners.rst) to build and start an active learning loop;
 - Use the [Argilla UI](../reference/webapp/annotate_records.md) to annotate examples and learn actively;

![raining-textclassification-smalltext-activelearning](../../_static/tutorials/training-textclassification-smalltext-activelearning/training-textclassification-smalltext-activelearning.png)

## Introduction

> Active learning is a special case of machine learning in which a learning algorithm can interactively query a user (or some other information source) to label new data points with the desired outputs. [Wikipedia](https://en.wikipedia.org/wiki/Active_learning_(machine_learning))

Supervised machine learning often requires large amounts of labeled data that are expensive to generate. 
*Active Learning* (AL) systems attempt to overcome this labeling bottleneck. 
The underlying idea is that not all data points are equally important for training the model. 
The AL system tries to query only the most relevant data from a pool of unlabeled data to be labeled by a so-called *oracle*, which is often a human annotator.
Therefore, AL systems are usually much more sample efficient and need far less training data than traditional supervised systems.

This tutorial will show you how to incorporate [Argilla](https://github.com/argilla-io/argilla) into an active learning workflow involving a human in the loop.
We will build a simple text classifier by combining the active learning framework [small-text](https://github.com/webis-de/small-text) and Argilla. 
Hugging Face's [transformers](https://github.com/huggingface/transformers) will provide the classifier we will embed in an active learner from small-text. Argilla excels in making **you** the oracle that conveniently teaches the model via an intuitive UI.

## Setup

For this tutorial, we also need some optional and third-party libraries that you can install via pip:

In [ ]:
%pip install "argilla[listeners]" "datasets~=2.5.0" "small-text>=1.1.0"  "transformers[torch]"

## The TREC dataset

For this tutorial, we will use the well-known [TREC dataset](https://huggingface.co/datasets/trec) containing 6000 labeled questions; 5500 in the training and 500 in the test split. 
This dataset can be turned into a text classification task, in which a model must predict one of the six coarse labels given the question. 
The labels consist of ABBREVIATION (ABBR), ENTITY (ENTY), DESCRIPTION (DESC), HUMAN (HUM), LOCATION (LOC), and NUMERIC VALUE (NUM).

Let us load the dataset from the [Hugging Face Hub](https://huggingface.co/datasets):

In [ ]:
import datasets

trec = datasets.load_dataset("trec", version=datasets.Version("2.0.0"))


## Preprocessing the dataset

We first need to wrap the dataset in a specific data class provided by [small-text](https://github.com/webis-de/small-text), the excellent active learning framework we will use in this tutorial. 
Since we will choose a [Hugging Face transformer](https://huggingface.co/docs/transformers/index) in the active learner, small-text will expect a `TransformersDataset` object that already contains the tokenized input text.

In order to build a `TransformersDataset` object, we first need a tokenizer:

In [ ]:
import torch
from transformers import AutoTokenizer

# Choose transformer model: In non-gpu environments we use a tiny model to reduce the runtime
if not torch.cuda.is_available():
    transformer_model = "prajjwal1/bert-tiny"
else:
    transformer_model = "bert-base-uncased"

# Init tokenizer
tokenizer = AutoTokenizer.from_pretrained(transformer_model)


With this, we can create a `TransformersDataset` by calling `TransformersDataset.from_arrays()` which expects a list of texts, a numpy array (which indicates single-label classification), a tokenizer, and lastly the possible (integer values) of target labels within this dataset.

In [ ]:
import numpy as np
from small_text import TransformersDataset

num_classes = trec["train"].features["coarse_label"].num_classes
target_labels = np.arange(num_classes)

train_text = [row["text"] for row in trec["train"]]
train_labels = np.array([row["coarse_label"] for row in trec["train"]])

# Create the dataset for small-text
dataset = TransformersDataset.from_arrays(
    train_text, train_labels, tokenizer, target_labels=target_labels
)


We will also create a test dataset to track the performance of the transformer model during the active learning loop.

In [ ]:
# Create test dataset
test_text = [row["text"] for row in trec["test"]]
test_labels = np.array([row["coarse_label"] for row in trec["test"]])

dataset_test = TransformersDataset.from_arrays(
    test_text, test_labels, tokenizer, target_labels=np.arange(num_classes)
)


## Setting up the active learner

Now that we have our datasets ready let's set up the active learner. 
For this, we need two components:

 - the **classifier** to be trained;
 - the **query strategy** to obtain the most relevant data;
 
In our case, we choose a [Hugging Face transformer](https://huggingface.co/docs/transformers/index) as the classifier and a [tie-breaker](https://small-text.readthedocs.io/en/v1.0.0/components/query_strategies.html#small_text.query_strategies.strategies.BreakingTies) as the query strategy. 
The latter selects instances of the data pool with a small margin between the two most likely predicted labels. 

In [ ]:
import logging

from small_text import (
    BreakingTies,
    PoolBasedActiveLearner,
    TransformerBasedClassificationFactory,
    TransformerModelArguments,
)

# Set logger to get training feedback
logging.getLogger("small_text").setLevel(logging.INFO)

# Define our classifier
clf_factory = TransformerBasedClassificationFactory(
    TransformerModelArguments(transformer_model),
    num_classes=6,
    kwargs={"verbosity": 100}
    # If you have a cuda device, specify it here.
    # Otherwise, just remove the following line.
    # kwargs={"device": "cuda"}
)

# Define our query strategy
query_strategy = BreakingTies()

# Use the active learner with a pool containing all unlabeled data
active_learner = PoolBasedActiveLearner(clf_factory, query_strategy, dataset)


Since most query strategies, including ours, require a trained model, we randomly draw a subset from the data pool to initialize our AL system. 
After obtaining the labels for this batch of instances, the active learner will use them to create the first classifier.


In [ ]:
from small_text import random_initialization
import numpy as np

# Fix seed for reproducibility
np.random.seed(42)


# Number of samples in our queried batches
NUM_SAMPLES = 20

# Randomly draw an initial subset from the data pool
initial_indices = random_initialization(dataset, NUM_SAMPLES)


## Argilla and you: the perfect oracle

With our active learner ready, it is time to teach it. 
We first create a [Argilla dataset](../reference/webapp/dataset.md) to log and label the initial random batch queried by the active learner.

In [ ]:
import argilla as rg

# Choose a name for the dataset
DATASET_NAME = "trec_with_active_learning"

# Define labeling schema
labels = trec["train"].features["coarse_label"].names
settings = rg.TextClassificationSettings(label_schema=labels)

# Create dataset with a label schema
rg.configure_dataset(name=DATASET_NAME, settings=settings)

# Create records from the initial batch
records = [
    rg.TextClassificationRecord(
        text=trec["train"]["text"][idx],
        metadata={"batch_id": 0},
        id=idx,
    )
    for idx in initial_indices
]

# Log initial records to Argilla
rg.log(records, DATASET_NAME)


Before switching to the Argilla UI to label the records, we will set up the **active learning loop**. 
For this, we will use the [listener decorator](../reference/python/python_listeners.rst#argilla.listeners.listener) from Argilla. 
The loop will run automatically once all records of a batch are labeled (see the `query` and `condition` argument of the decorator). 
It will trigger the classifier's training, query a new batch from the active learner and log it to Argilla. 
We will also keep track of the accuracy of the current classifier by evaluating it on our test set.

In [ ]:
from argilla.listeners import listener
from sklearn.metrics import accuracy_score

# Define some helper variables
LABEL2INT = trec["train"].features["coarse_label"].str2int
ACCURACIES = []

# Set up the active learning loop with the listener decorator
@listener(
    dataset=DATASET_NAME,
    query="status:Validated AND metadata.batch_id:{batch_id}",
    condition=lambda search: search.total == NUM_SAMPLES,
    execution_interval_in_seconds=3,
    batch_id=0,
)
def active_learning_loop(records, ctx):
    # 1. Update active learner
    print(f"Updating with batch_id {ctx.query_params['batch_id']} ...")
    y = np.array([LABEL2INT(rec.annotation) for rec in records])

    # initial update
    print(
        "Starting training! Depending on you device, this might take a while, so set up"
        " logging."
    )
    if ctx.query_params["batch_id"] == 0:
        indices = np.array([rec.id for rec in records])
        active_learner.initialize_data(indices, y)
    # update with the prior queried indices
    else:
        active_learner.update(y)
    print("Done!")

    # 2. Query active learner
    print("Querying new data points ...")
    queried_indices = active_learner.query(num_samples=NUM_SAMPLES)
    new_batch = ctx.query_params["batch_id"] + 1
    new_records = [
        rg.TextClassificationRecord(
            text=trec["train"]["text"][idx],
            metadata={"batch_id": new_batch},
            id=idx,
        )
        for idx in queried_indices
    ]

    # 3. Log the batch to Argilla
    rg.log(new_records, DATASET_NAME)

    # 4. Evaluate current classifier on the test set
    print("Evaluating current classifier ...")
    accuracy = accuracy_score(
        dataset_test.y,
        active_learner.classifier.predict(dataset_test),
    )

    ACCURACIES.append(accuracy)
    ctx.query_params["batch_id"] = new_batch
    print("Done!")

    print("Waiting for annotations ...")


## Starting the active learning loop

Now we can start the loop and switch to the Argilla UI.

In [ ]:
active_learning_loop.start()


In the Argilla UI, we will set the number of records per page to 20 since it is also our chosen batch size. 
Furthermore, we will use the [Status filter](../reference/webapp/filter_records.md) to filter out already annotated records. 
Now, all we have to do is to annotate the displayed records. 
Once annotating everything, the classifier's training will be automatically triggered.

After a few seconds, you should see the newly queried batch when pressing the [Refresh button](../reference/webapp/dataset.md#refresh). 
The training can take longer depending on your machine and whether you have a CUDA device. 
You can always check the status of the active learning loop from your notebook.

## Can we stop?

After a few iterations, we can check the accuracy of the current classifier and plot its evaluation.

In [ ]:
import pandas as pd

pd.Series(ACCURACIES).plot(xlabel="Iteration", ylabel="Accuracy")


We should achieve an accuracy of at least **0.8 after around 12 iterations**, corresponding to roughly 260 annotated records. 
The stopping criterion is ultimately up to you, and you can choose more sophisticated criteria like the [KappaAverage](https://small-text.readthedocs.io/en/v1.0.0/components/stopping_criteria.html) implemented in small-text.

In [ ]:
active_learning_loop.stop()


## Summary

In this tutorial, we saw how you could **embed Argilla in an active learning loop involving a human in the loop**. 
We relied on **small-text to use a Hugging Face transformer within an active learning setup**. 
In the end, we gathered **a sample-efficient data set by annotating only the most informative records** for the model.

Argilla makes it very easy to use a dedicated annotation team or subject matter experts as an oracle for your active learning system. They will only interact with the Argilla UI and do not have to worry about training or querying the system. We encourage you to try out active learning in your next project and make your and your annotator's life a little easier.

## Next steps

⭐ [Argilla Github repo](https://github.com/argilla-io/argilla) to stay updated.

📚 [Argilla documentation](https://argilla.readthedocs.io/en/stable/index.html) for more guides and tutorials.

🙋‍♀️ Join the Argilla community! A good place to start is our [slack channel](https://join.slack.com/t/rubrixworkspace/shared_invite/zt-whigkyjn-a3IUJLD7gDbTZ0rKlvcJ5g).
